In [1]:
import numpy as np
import os
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50,preprocess_input
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
classes = os.listdir("/home/vasu/all_projects/CB/data_set/")
print classes

['dogs', 'cats', 'horses', 'Humans']


In [3]:
dict_labels = {'dogs':0, 'cats':1, 'horses':2, 'Humans':3}

In [4]:
path = "/home/vasu/all_projects/CB/data_set/"

In [5]:
x_,y_ = [],[]
for ix in classes:
    img_list = os.listdir(path+ix)
    for img in img_list:
        img_path = path + ix + "/" + img
        x = image.load_img(img_path, target_size=(224,224))
        x = image.img_to_array(x)
        x = x/255.0
        x_.append(x)
        y_.append(dict_labels[ix])

In [6]:
x = np.array(x_)
y = np.array(y_)
y = np_utils.to_categorical(y)
print x.shape,y.shape

(808, 224, 224, 3) (808, 4)


In [7]:
split = int(0.8*(x.shape[0]))
X_train = x[:split,:]
X_test = x[split:,:]
Y_train = y[:split,:]
Y_test = y[split:,:]
print X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(646, 224, 224, 3) (646, 4) (162, 224, 224, 3) (162, 4)


In [8]:
image_input = Input(shape=(224,224,3))

model = ResNet50(input_tensor=image_input, include_top=True, weights='imagenet')

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [10]:
last_layer = model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128,activation='relu',name='fc1')(x)
out = Dense(4, activation='softmax', name='output')(x)

custom_resnet = Model(image_input,out)

for layer in custom_resnet.layers[:-3]:
    layer.trainable = False

In [11]:
custom_resnet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 112, 112, 64)  9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           bn_conv1[0][0]                   
___________________________________________________________________________________________

In [12]:
custom_resnet.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
custom_resnet.fit(X_train,Y_train,epochs=2,shuffle=True,batch_size=128,validation_data=(X_test,Y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/2
646/646 [==============================] - 67s - loss: 0.0934 - acc: 0.9814 - val_loss: 1.1550 - val_acc: 0.0000e+00
Epoch 2/2
646/646 [==============================] - 67s - loss: 0.0372 - acc: 0.9938 - val_loss: 1.4596 - val_acc: 0.0000e+00
